In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Roraima - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [12]:
data = pd.read_csv('2003_01_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Roraima - Produção de Cimento (t)'], axis=1)
data

,Ano,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Consumo de Cimento (t)
0,2003,8.263084,3.596068,0.734916,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,6.128460e+06,332580.202821,12.101540,5.339136e+06,74.283000
1,2004,8.191545,3.429895,0.735404,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,6.226406e+06,336594.835486,12.124873,5.374190e+06,39.775000
2,2005,8.166961,3.071663,0.735800,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,6.321383e+06,339978.313869,12.165244,5.409179e+06,30.179000
3,2006,8.072306,3.045251,0.736105,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,6.410128e+06,343100.125013,12.200109,5.439894e+06,34.243000
4,2007,8.047939,3.353816,0.736322,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,6.486417e+06,340068.183172,12.212555,5.458573e+06,35.931000
5,2008,8.015090,3.689138,0.736454,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,6.555833e+06,342849.112404,12.235418,5.474033e+06,60.009000
6,2009,8.046321,3.874501,0.736503,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,6.611657e+06,345335.485288,12.228462,5.482306e+06,63.064000
7,2010,8.009229,4.024163,0.736471,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,6.787712e+06,348962.483930,12.215749,5.534487e+06,86.408000
8,2011,7.996819,4.171630,0.736360,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,6.947966e+06,347630.972013,12.206783,5.574553e+06,103.009000
9,2012,8.651837,4.318665,0.736089,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,7.096137e+06,347411.232273,12.186539,5.609510e+06,100.066000


In [13]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
0,8.263084,3.596068,0.734916,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,6.128460e+06,332580.202821,12.101540,5.339136e+06
1,8.191545,3.429895,0.735404,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,6.226406e+06,336594.835486,12.124873,5.374190e+06
2,8.166961,3.071663,0.735800,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,6.321383e+06,339978.313869,12.165244,5.409179e+06
3,8.072306,3.045251,0.736105,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,6.410128e+06,343100.125013,12.200109,5.439894e+06
4,8.047939,3.353816,0.736322,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,6.486417e+06,340068.183172,12.212555,5.458573e+06
5,8.015090,3.689138,0.736454,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,6.555833e+06,342849.112404,12.235418,5.474033e+06
6,8.046321,3.874501,0.736503,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,6.611657e+06,345335.485288,12.228462,5.482306e+06
7,8.009229,4.024163,0.736471,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,6.787712e+06,348962.483930,12.215749,5.534487e+06
8,7.996819,4.171630,0.736360,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,6.947966e+06,347630.972013,12.206783,5.574553e+06
9,8.651837,4.318665,0.736089,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,7.096137e+06,347411.232273,12.186539,5.609510e+06


In [14]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      39.775000
1      30.179000
2      34.243000
3      35.931000
4      60.009000
5      63.064000
6      86.408000
7     103.009000
8     100.066000
9     121.914000
10    120.417571
11    113.282057
12    105.481371
13     99.723000
14     83.012000
15    121.120000
16    151.571000
17    144.989000
18    132.217000
19           NaN
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [15]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
0,-0.709627,-0.806714,-0.174694,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.590661,-2.240264,-1.209834,-1.737333
1,-0.771439,-1.021390,0.118862,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.395560,-1.319744,-0.740358,-1.464888
2,-0.792681,-1.484184,0.356742,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.206374,-0.543941,0.071913,-1.192940
3,-0.874466,-1.518306,0.540358,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.029601,0.171864,0.773399,-0.954220
4,-0.895520,-1.119675,0.671120,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.877638,-0.523335,1.023829,-0.809039
5,-0.923903,-0.686479,0.750440,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.739368,0.114309,1.483848,-0.688880
6,-0.896919,-0.447012,0.779730,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.628171,0.684413,1.343888,-0.624586
7,-0.928968,-0.253666,0.760401,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.277482,1.516052,1.088083,-0.219018
8,-0.939690,-0.063156,0.693865,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.041732,1.210748,0.907693,0.092385
9,-0.373730,0.126796,0.530420,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.336878,1.160364,0.500379,0.364075


In [16]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      39.775000
1      30.179000
2      34.243000
3      35.931000
4      60.009000
5      63.064000
6      86.408000
7     103.009000
8     100.066000
9     121.914000
10    120.417571
11    113.282057
12    105.481371
13     99.723000
14     83.012000
15    121.120000
16    151.571000
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [17]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
17,0.674508,2.184119,-2.88705,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.943905,-0.349474,-1.517602,0.929344


In [18]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    144.989
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [19]:
train_input.iloc[-(len(train_input)//5):]

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
14,1.252131,1.123678,-0.943913,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.261385,-0.537740,-0.929289,1.250944
15,1.346017,1.319327,-1.619925,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.271596,-0.450795,-1.442269,1.273228
16,1.162902,1.813945,-3.122943,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.180119,-0.498884,-1.686584,1.168121


In [20]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [21]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [23]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3725516238, 3815781321, 2429468138, 2065346453, 1370707402, 2382476725, 3080286255, 1741280190, 2758383211, 3937494254, 356775603, 1445131682, 1935923717, 4206347050, 2367617419, 3039639798, 1351659272, 1485321368, 3575766829, 3901312171, 3638609303, 3352967061, 3692545300, 225429519, 1599769097, 2007520823, 1496661728, 614866591, 1148831180, 2713972559, 302325614, 2259986207, 1986091049, 93399519, 2778080528, 742574948, 762599657, 2574912090, 2182085923, 1296832707, 3601171531, 1857916579, 3866805574, 2647979448, 879701404, 2822403134, 2056203949, 1362203292, 3876817530, 106903010, 2512230447, 1712043522, 355533583, 2037583719, 3745578569, 316888400, 4066747614, 612094099, 77209735, 2721917511, 648745340, 1959395472, 2440372657, 1415335104, 1123051944, 3886086566, 2499601164, 502011071, 2777945916, 1253510962, 1804177304, 1331193905, 3347823876, 2345968021, 2840853529, 467816574, 1653394220, 662622249, 3058298528, 1081896151, 594966911, 3059878827, 3732470621, 1614410771, 835539329, 

val_loss: 786.6921997070312


Step: 84 ___________________________________________
val_loss: 783.7750854492188


Step: 85 ___________________________________________
val_loss: 0.0011244944762438536


Step: 86 ___________________________________________
val_loss: 799.4296875


Step: 87 ___________________________________________
val_loss: 813.98876953125


Step: 88 ___________________________________________
val_loss: 764.0333862304688


Step: 89 ___________________________________________
val_loss: 197.7783660888672


Step: 90 ___________________________________________
val_loss: 85.35330200195312


Step: 91 ___________________________________________
val_loss: 205.6189422607422


Step: 92 ___________________________________________
val_loss: 808.4100952148438


Step: 93 ___________________________________________
val_loss: 0.00044686731416732073


Step: 94 ___________________________________________
val_loss: 0.5783342719078064


Step: 95 ___________________________________________
va

In [24]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 452ms/step - loss: 8724.5684 - val_loss: 12204.4961
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 7455.0273 - val_loss: 15443.3115
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 5016.4282 - val_loss: 3695.7012
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 5434.4824 - val_loss: 4798.7383
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 5226.0044 - val_loss: 4849.7378
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 4093.6782 - val_loss: 36130.4688
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 12478.2578 - val_loss: 12227.0029
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 8016.3833 - val_loss: 14126.3877
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 7667.9224 - val_loss: 13884.1768
Epoch 10/10000
1/1 [==================

1/1 [==============================] - 0s 14ms/step - loss: 988.7911 - val_loss: 1247.3687
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 984.3547 - val_loss: 1239.9211
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 997.7363 - val_loss: 1274.6595
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 980.6952 - val_loss: 1251.4913
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 977.4335 - val_loss: 1229.1642
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 963.6434 - val_loss: 1245.0619
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 957.4487 - val_loss: 1238.1862
Epoch 84/10000
1/1 [==============================] - 0s 13ms/step - loss: 950.9273 - val_loss: 1235.5758
Epoch 85/10000
1/1 [==============================] - 0s 14ms/step - loss: 939.9808 - val_loss: 1226.3535
Epoch 86/10000
1/1 [==============================] - 0s 13ms

Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 624.2745 - val_loss: 1106.2543
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 620.8171 - val_loss: 1106.4436
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 617.0269 - val_loss: 1099.9136
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 613.8779 - val_loss: 1101.6373
Epoch 158/10000
1/1 [==============================] - 0s 13ms/step - loss: 610.3240 - val_loss: 1097.7811
Epoch 159/10000
1/1 [==============================] - 0s 13ms/step - loss: 606.9419 - val_loss: 1095.7592
Epoch 160/10000
1/1 [==============================] - 0s 13ms/step - loss: 603.7113 - val_loss: 1093.1263
Epoch 161/10000
1/1 [==============================] - 0s 13ms/step - loss: 600.4526 - val_loss: 1090.9003
Epoch 162/10000
1/1 [==============================] - 0s 13ms/step - loss: 597.2156 - val_loss: 1093.3857
Epoch 163/10000
1/1 [================

1/1 [==============================] - 0s 13ms/step - loss: 427.4087 - val_loss: 1009.1376
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 427.1093 - val_loss: 1002.6538
Epoch 232/10000
1/1 [==============================] - 0s 13ms/step - loss: 424.2149 - val_loss: 994.5345
Epoch 233/10000
1/1 [==============================] - 0s 14ms/step - loss: 422.0232 - val_loss: 1010.2794
Epoch 234/10000
1/1 [==============================] - 0s 13ms/step - loss: 420.0393 - val_loss: 1006.0449
Epoch 235/10000
1/1 [==============================] - 0s 13ms/step - loss: 459.4830 - val_loss: 1063.6451
Epoch 236/10000
1/1 [==============================] - 0s 13ms/step - loss: 418.1254 - val_loss: 1009.9211
Epoch 237/10000
1/1 [==============================] - 0s 14ms/step - loss: 412.0759 - val_loss: 966.4418
Epoch 238/10000
1/1 [==============================] - 0s 14ms/step - loss: 414.5874 - val_loss: 1009.6418
Epoch 239/10000
1/1 [==============================] - 

1/1 [==============================] - 0s 13ms/step - loss: 312.7367 - val_loss: 941.9199
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 312.1620 - val_loss: 920.2957
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 316.8127 - val_loss: 953.7957
Epoch 310/10000
1/1 [==============================] - 0s 13ms/step - loss: 314.3138 - val_loss: 951.9968
Epoch 311/10000
1/1 [==============================] - 0s 14ms/step - loss: 313.3456 - val_loss: 952.0671
Epoch 312/10000
1/1 [==============================] - 0s 13ms/step - loss: 312.2178 - val_loss: 947.8304
Epoch 313/10000
1/1 [==============================] - 0s 13ms/step - loss: 310.7974 - val_loss: 943.5974
Epoch 314/10000
1/1 [==============================] - 0s 14ms/step - loss: 309.9987 - val_loss: 947.3076
Epoch 315/10000
1/1 [==============================] - 0s 13ms/step - loss: 308.6540 - val_loss: 944.5954
Epoch 316/10000
1/1 [==============================] - 0s 14ms

...

1/1 [==============================] - 0s 13ms/step - loss: 0.9470 - val_loss: 0.0085
Epoch 3127/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.9139 - val_loss: 0.0055
Epoch 3128/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9128 - val_loss: 0.0089
Epoch 3129/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9140 - val_loss: 0.0071
Epoch 3130/10000
1/1 [==============================] - 0s 13ms/step - loss: 1.0729 - val_loss: 0.0107
Epoch 3131/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9309 - val_loss: 0.0052
Epoch 3132/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9149 - val_loss: 8.9213e-04
Epoch 3133/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9105 - val_loss: 0.0011
Epoch 3134/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9097 - val_loss: 0.0011
Epoch 3135/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9083 - 

1/1 [==============================] - 0s 13ms/step - loss: 0.9099 - val_loss: 5.3750e-04
Epoch 3205/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9085 - val_loss: 2.9490e-04
Epoch 3206/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9095 - val_loss: 6.3925e-04
Epoch 3207/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9221 - val_loss: 0.0058
Epoch 3208/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9095 - val_loss: 0.0027
Epoch 3209/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9070 - val_loss: 9.3374e-04
Epoch 3210/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9143 - val_loss: 7.6215e-04
Epoch 3211/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9091 - val_loss: 9.3087e-04
Epoch 3212/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.8983 - val_loss: 4.4773e-04
Epoch 3213/10000
1/1 [==============================] - 0s 14m

1/1 [==============================] - 0s 13ms/step - loss: 0.8991 - val_loss: 2.1380e-04
Epoch 3283/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9077 - val_loss: 1.4117e-04
Epoch 3284/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9058 - val_loss: 7.1604e-04
Epoch 3285/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9024 - val_loss: 0.0016
Epoch 3286/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8845 - val_loss: 0.0019
Epoch 3287/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9045 - val_loss: 6.8478e-04
Epoch 3288/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8647 - val_loss: 0.0016
Epoch 3289/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9041 - val_loss: 1.0895e-04
Epoch 3290/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9002 - val_loss: 9.2871e-05
Epoch 3291/10000
1/1 [==============================] - 0s 13ms/st

1/1 [==============================] - 0s 13ms/step - loss: 0.9060 - val_loss: 0.0021
Epoch 3361/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9059 - val_loss: 0.0034
Epoch 3362/10000
1/1 [==============================] - 0s 14ms/step - loss: 0.9131 - val_loss: 0.0067
Epoch 3363/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9084 - val_loss: 0.0019
Epoch 3364/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9058 - val_loss: 0.0011
Epoch 3365/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9095 - val_loss: 4.4687e-04
Epoch 3366/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9063 - val_loss: 1.9019e-04
Epoch 3367/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9135 - val_loss: 8.3837e-04
Epoch 3368/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9272 - val_loss: 6.6456e-05
Epoch 3369/10000
1/1 [==============================] - 0s 13ms/step - los

1/1 [==============================] - 0s 14ms/step - loss: 0.9085 - val_loss: 2.3604e-04
Epoch 3438/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9081 - val_loss: 0.0032
Epoch 3439/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.9071 - val_loss: 5.9176e-04
Epoch 3440/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8682 - val_loss: 0.0038
Epoch 3441/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8355 - val_loss: 0.0018
Epoch 3442/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8895 - val_loss: 2.2826e-04
Epoch 3443/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8566 - val_loss: 0.0027
Epoch 3444/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.8152 - val_loss: 8.4620e-04
Epoch 3445/10000
1/1 [==============================] - 0s 13ms/step - loss: 0.7977 - val_loss: 8.0632e-04
Epoch 3446/10000
1/1 [==============================] - 0s 13ms/step -

In [25]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[144.989](test_target) - [[125.96028]](prediction) = 19.028718627929692


In [26]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [27]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
0,1.846009,0.258449,-1.987494,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.554040,-1.803918,-1.803626,-1.697712
1,1.028419,-0.243821,-1.089980,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.067021,-0.962149,-1.302342,-1.101641
2,0.747451,-1.326605,-0.362689,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.594770,-0.252717,-0.435038,-0.506657
3,-0.334325,-1.406437,0.198693,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.153504,0.401849,0.313976,0.015627
4,-0.612806,-0.473775,0.598482,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.225831,-0.233874,0.581373,0.333264
5,-0.988232,0.539760,0.840994,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.570985,0.349218,1.072560,0.596153
6,-0.631301,1.100033,0.930545,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.848560,0.870549,0.923117,0.736820
7,-1.055214,1.552396,0.871449,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.723959,1.631041,0.649980,1.624145


train_target:


,Roraima - Consumo de Cimento (t)
0,39.775
1,30.179
2,34.243
3,35.931
4,60.009
5,63.064
6,86.408
7,103.009


test_input:


...

,Roraima - Desemprego,Roraima - Valor Cimento Portland,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado
17,0.674508,2.184119,-2.88705,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.943905,-0.349474,-1.517602,0.929344


test_target:


,Roraima - Consumo de Cimento (t)
17,144.989


1/1 [==============================] - 0s 34ms/step
Error: 19.028718627929692




[70.8981704711914,
 99.18984985351562,
 122.4375228881836,
 87.48020935058594,
 113.27665710449219,
 111.67495727539062,
 99.72689819335938,
 83.1805648803711,
 103.36026763916016,
 125.96028137207031]

In [28]:
display(targets)
display(predictions)

[100.066,
 121.914,
 120.41757142857142,
 113.28205714285713,
 105.48137142857142,
 99.723,
 83.012,
 121.12,
 151.571,
 144.989]

[70.8981704711914,
 99.18984985351562,
 122.4375228881836,
 87.48020935058594,
 113.27665710449219,
 111.67495727539062,
 99.72689819335938,
 83.1805648803711,
 103.36026763916016,
 125.96028137207031]

In [29]:
mae = mean_absolute_error(predictions, targets)
mae

22.135480618024552

In [30]:
porcentage = mae/np.mean(targets)
porcentage

0.19056420430539672